# <div align="center" style="color: #ff5733;">App Categorization</div>

**Created By**  - DWAIPAYAN CHAKROBORTI

**Project** - App Categorization

**Requested By** - Biswa

**Date** - 01-10-2024

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
client = bigquery.Client(project='prj-prod-dataplatform')

In [ ]:
# Run the entire query

sq = """WITH
  b AS (
  SELECT
    loanAccountNumber,
    min_inst_def30,
    obs_min_inst_def30
  FROM
    prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data
  WHERE
    obs_min_inst_def30 >= 2),
lmt as
(SELECT
  lmt.loanAccountNumber,
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  case when b.obs_min_inst_def30 >=2 and b.min_inst_def30 in (1,2) then lmt.loanAccountNumber end FSPD30_loancnt,
  case when b.obs_min_inst_def30 >=2 then lmt.loanAccountNumber end obsFSPD30_loancnt
FROM
  `risk_credit_mis.loan_master_table` lmt
left JOIN
  b
ON
  lmt.loanAccountNumber = b.loanAccountNumber 
),
base as
(select 
distinct
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  t3.creditScoreUpdated   ,
  t3.fraudScore   ,	
  t3.fraudScoreUpdated    ,
  t3.calculateddate   ,
  t4.run_date ,
  ca.package_name ,
  ca.first_install_time    ,
  ca.last_update_time      ,
  ca.version_name, 
  ca.version_code,
  t4.GeneralInfo.brand     ,
  t4.Hardware.device__brand   ,
  t4.Hardware.device__manufacturer   ,
  t4.Hardware.device__model,
  t4.GeneralData.telephony_info__network_operator_name,
  t4.GeneralData.telephony_info__network_operator,
  t4.GeneralData.sim_operator_name,
  ptat.Category,
  -- ptat.Rating,
  case when ptat.Rating = 'rated for 3+' then 1 else 0 end rated_for_3_plus,
  case when ptat.Rating = 'rated for 7+' then 1 else 0 end rated_for_7_plus,
  case when ptat.Rating = 'rated for 12+' then 1 else 0 end rated_for_12_plus,
  case when ptat.Rating = 'rated for 16+' then 1 else 0 end rated_for_16_plus,
  case when ptat.Rating = 'rated for 18+' then 1 else 0 end rated_for_18_plus,
  case when ptat.Rating = 'undefined' then 1 else 0 end undefined,
  case when ptat.Rating = 'unrated' then 1 else 0 end unrated,
  case when ptat.Rating is null then 1 else 0 end Rating_Not_Available,
  ptat.Is_Paid,

  lmt.FSPD30_loancnt,     ---- FSPD30 = 1 when this value is not null(provided this as there were be duplicate rows in this dataset because of package name)
  lmt.obsFSPD30_loancnt   ---- obsFSPD30 = 1 when this value is not null (provided this as there were be duplicate rows in this dataset because of package name)
from lmt
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON lmt.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
inner join
`prj-prod-dataplatform.core_raw.loan_accounts` loan
on loan.CUSTOMERID = lmt.customerId
 INNER JOIN
(select deviceId, af.package_name as package_name, af.first_install_time as first_install_time , af.last_update_time as last_update_time 
, version_name, version_code
from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
unnest(Application) as af) ca
ON ca.deviceId = t3.refno
LEFT JOIN prj-prod-dataplatform.dap_ds_poweruser_playground.PH_Tonikbank_Application_Temp ptat
ON REGEXP_REPLACE(ca.package_name, r'[ ._]', '') = REGEXP_REPLACE(ptat.Package_Name, r'[ ._]', '')
where date(lmt.startApplyDateTime) >='2023-07-01'   ---- Please change the date as per your requirement. This is Loan Application Apply Date
-- and lmt.FSPD30_loancnt is not null
)
select * from base 

;"""

df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Query executing stage S01: Input and status COMPLETE : 30.02s:   6%|▋         |

In [ ]:
df.columns

In [ ]:
df.shape

Get all distinct package names and their corresponding frequencies and sort them in the descending order of frequency

In [2]:
sq = """WITH
  b AS (
  SELECT
    loanAccountNumber,
    min_inst_def30,
    obs_min_inst_def30
  FROM
    prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data
  WHERE
    obs_min_inst_def30 >= 2),
lmt as
(SELECT
  lmt.loanAccountNumber,
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  case when b.obs_min_inst_def30 >=2 and b.min_inst_def30 in (1,2) then lmt.loanAccountNumber end FSPD30_loancnt,
  case when b.obs_min_inst_def30 >=2 then lmt.loanAccountNumber end obsFSPD30_loancnt
FROM
  `risk_credit_mis.loan_master_table` lmt
left JOIN
  b
ON
  lmt.loanAccountNumber = b.loanAccountNumber 
),
base as
(select 
distinct
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  t3.creditScoreUpdated   ,
  t3.fraudScore   ,	
  t3.fraudScoreUpdated    ,
  t3.calculateddate   ,
  t4.run_date ,
  ca.package_name ,
  ca.first_install_time    ,
  ca.last_update_time      ,
  ca.version_name, 
  ca.version_code,
  t4.GeneralInfo.brand     ,
  t4.Hardware.device__brand   ,
  t4.Hardware.device__manufacturer   ,
  t4.Hardware.device__model,
  t4.GeneralData.telephony_info__network_operator_name,
  t4.GeneralData.telephony_info__network_operator,
  t4.GeneralData.sim_operator_name,
  ptat.Category,
  -- ptat.Rating,
  case when ptat.Rating = 'rated for 3+' then 1 else 0 end rated_for_3_plus,
  case when ptat.Rating = 'rated for 7+' then 1 else 0 end rated_for_7_plus,
  case when ptat.Rating = 'rated for 12+' then 1 else 0 end rated_for_12_plus,
  case when ptat.Rating = 'rated for 16+' then 1 else 0 end rated_for_16_plus,
  case when ptat.Rating = 'rated for 18+' then 1 else 0 end rated_for_18_plus,
  case when ptat.Rating = 'undefined' then 1 else 0 end undefined,
  case when ptat.Rating = 'unrated' then 1 else 0 end unrated,
  case when ptat.Rating is null then 1 else 0 end Rating_Not_Available,
  ptat.Is_Paid,

  lmt.FSPD30_loancnt,     ---- FSPD30 = 1 when this value is not null(provided this as there were be duplicate rows in this dataset because of package name)
  lmt.obsFSPD30_loancnt   ---- obsFSPD30 = 1 when this value is not null (provided this as there were be duplicate rows in this dataset because of package name)
from lmt
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON lmt.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
inner join
`prj-prod-dataplatform.core_raw.loan_accounts` loan
on loan.CUSTOMERID = lmt.customerId
 INNER JOIN
(select deviceId, af.package_name as package_name, af.first_install_time as first_install_time , af.last_update_time as last_update_time 
, version_name, version_code
from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
unnest(Application) as af) ca
ON ca.deviceId = t3.refno
LEFT JOIN prj-prod-dataplatform.dap_ds_poweruser_playground.PH_Tonikbank_Application_Temp ptat
ON REGEXP_REPLACE(ca.package_name, r'[ ._]', '') = REGEXP_REPLACE(ptat.Package_Name, r'[ ._]', '')
where date(lmt.startApplyDateTime) >='2023-07-01'   ---- Please change the date as per your requirement. This is Loan Application Apply Date
-- and lmt.FSPD30_loancnt is not null
)
select package_name, count(distinct customerId) cntcust 
, (select count(distinct customerId) from base) totalcust
from base group by 1 order by 2 desc

;"""

In [3]:
dfpackages = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 92747bf7-4ef3-4612-90d8-b7da7f346438 successfully executed: 100%|██████████|

I0000 00:00:1727757609.137722   28868 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache, work_serializer_dispatch
I0000 00:00:1727757609.137985   28868 ev_epoll1_linux.cc:125] grpc epoll fd: 59



Downloading: 100%|██████████|


I0000 00:00:1727757610.140923   28914 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1727757610.142046   28914 work_stealing_thread_pool.cc:269] WorkStealingThreadPoolImpl::Quiesce


In [4]:
dfpackages['share'] = round(dfpackages['cntcust']/dfpackages['totalcust'] *100, 4)

In [5]:
dfpackages.sort_values(by = 'cntcust', ascending=False)

,package_name,cntcust,totalcust,share
0,com.tonik.mobile,117690,117690,100.0
1,android,117528,117690,99.8624
12,com.android.vpndialogs,117526,117690,99.8607
21,com.android.providers.telephony,117526,117690,99.8607
20,com.android.certinstaller,117526,117690,99.8607
...,...,...,...,...
90387,fr.dvilleneuve.lockito,1,117690,0.0008
90388,org.chromium.webapk.ab9743bfd65399748_v2,1,117690,0.0008
90389,org.chromium.webapk.abe15ed376e08338f_v2,1,117690,0.0008
90390,com.sosomod.sccpsmods,1,117690,0.0008


In [6]:
dfpackages.to_csv(r"/home/jupyter/DS_Team/DC/App_Package_Categorization/App_Categorization/Data/Distinctpackagenameavailableincredolabdata.csv", index = False)

In [7]:
def classify_package(row):
    package_name = row['package_name']
    share = row['share']
    
    # List of common system app prefixes
    system_prefixes = [
        'com.android.',
        'android',
        'com.google.android.',
        'com.sec.android.',  # Samsung
        'com.htc.',  # HTC
        'com.sonyericsson.',  # Sony
        'com.motorola.',  # Motorola
        'com.lge.',  # LG
    ]
    
    # List of known system apps that don't follow the common prefixes
    known_system_apps = [
        'com.google.android.gms',
        'com.google.android.gsf',
        'com.google.android.tts',
        'com.android.chrome',
        'com.google.android.apps.maps',
    ]
    
    # Check if the package name starts with any of the system prefixes
    if any(package_name.startswith(prefix) for prefix in system_prefixes):
        return 'System App'
    
    # Check if the package name is in the list of known system apps
    elif package_name in known_system_apps:
        return 'System App'
    
    # Check if the share is less than 1%
    elif share < 1.0:
        return 'Excluded App'
    
    # If it is tonik app then tonik app
    
    elif package_name == 'com.tonik.mobile':
        return 'Tonik App'
    
    # If it doesn't match any system app criteria, classify as a downloaded app
    else:
        return 'Downloaded App'

# Apply the function to the DataFrame
dfpackages['app_type'] = dfpackages.apply(classify_package, axis=1)


In [8]:
# dfpackages.drop(columns='is_system_app', inplace = True)

In [9]:
dfpackages.sample(30)

,package_name,cntcust,totalcust,share,app_type
26380,com.whiteapps.smartboosterpro,8,117690,0.0068,Excluded App
47601,com.android.nfcstm,3,117690,0.0025,System App
75745,com.flashworkshop.vr,1,117690,0.0008,Excluded App
11162,com.candle.piano_star,36,117690,0.0306,Excluded App
81383,com.gunsounds.gunapp.flashlight.gunsoundwithsh...,1,117690,0.0008,Excluded App
53951,coloring.animal.paint.drawing.color.book,2,117690,0.0017,Excluded App
132406,ph.visacenter.TDGTravel,1,117690,0.0008,Excluded App
43941,com.launcher.smart.diamond.heart.theme,3,117690,0.0025,Excluded App
84275,com.phlpay.winning.game,1,117690,0.0008,Excluded App
3878,com.coloros.keyguard.notification,265,117690,0.2252,Excluded App


In [10]:
dfpackages['app_type'].value_counts()

app_type
Excluded App      132945
System App          2674
Downloaded App      1864
Tonik App              1
Name: count, dtype: int64

In [11]:
dfpackages.describe()

,cntcust,totalcust,share
count,137484.0,137484.0,137484.0
mean,250.791765,117690.0,0.213062
std,3306.229678,0.0,2.809273
min,1.0,117690.0,0.0008
25%,1.0,117690.0,0.0008
50%,1.0,117690.0,0.0008
75%,5.0,117690.0,0.0042
max,117690.0,117690.0,100.0


In [12]:
sq = """WITH
  b AS (
  SELECT
    loanAccountNumber,
    min_inst_def30,
    obs_min_inst_def30
  FROM
    prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data
  WHERE
    obs_min_inst_def30 >= 2),
lmt as
(SELECT
  lmt.loanAccountNumber,
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  case when b.obs_min_inst_def30 >=2 and b.min_inst_def30 in (1,2) then lmt.loanAccountNumber end FSPD30_loancnt,
  case when b.obs_min_inst_def30 >=2 then lmt.loanAccountNumber end obsFSPD30_loancnt
FROM
  `risk_credit_mis.loan_master_table` lmt
left JOIN
  b
ON
  lmt.loanAccountNumber = b.loanAccountNumber 
),
base as
(select 
distinct
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.tsa_onboarding_time,
  lmt.startApplyDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.isTermsAndConditionsAccepted,
  lmt.disbursementDateTime,
  lmt.flagDisbursement,
  lmt.loanPaidStatus,
  t3.creditScoreUpdated   ,
  t3.fraudScore   ,	
  t3.fraudScoreUpdated    ,
  t3.calculateddate   ,
  t4.run_date ,
  ca.package_name ,
  ca.first_install_time    ,
  ca.last_update_time      ,
  ca.version_name, 
  ca.version_code,
  t4.GeneralInfo.brand     ,
  t4.Hardware.device__brand   ,
  t4.Hardware.device__manufacturer   ,
  t4.Hardware.device__model,
  t4.GeneralData.telephony_info__network_operator_name,
  t4.GeneralData.telephony_info__network_operator,
  t4.GeneralData.sim_operator_name,
  ptat.Category,
  -- ptat.Rating,
  case when ptat.Rating = 'rated for 3+' then 1 else 0 end rated_for_3_plus,
  case when ptat.Rating = 'rated for 7+' then 1 else 0 end rated_for_7_plus,
  case when ptat.Rating = 'rated for 12+' then 1 else 0 end rated_for_12_plus,
  case when ptat.Rating = 'rated for 16+' then 1 else 0 end rated_for_16_plus,
  case when ptat.Rating = 'rated for 18+' then 1 else 0 end rated_for_18_plus,
  case when ptat.Rating = 'undefined' then 1 else 0 end undefined,
  case when ptat.Rating = 'unrated' then 1 else 0 end unrated,
  case when ptat.Rating is null then 1 else 0 end Rating_Not_Available,
  ptat.Is_Paid,

  lmt.FSPD30_loancnt,     ---- FSPD30 = 1 when this value is not null(provided this as there were be duplicate rows in this dataset because of package name)
  lmt.obsFSPD30_loancnt   ---- obsFSPD30 = 1 when this value is not null (provided this as there were be duplicate rows in this dataset because of package name)
from lmt
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON lmt.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
inner join
`prj-prod-dataplatform.core_raw.loan_accounts` loan
on loan.CUSTOMERID = lmt.customerId
 INNER JOIN
(select deviceId, af.package_name as package_name, af.first_install_time as first_install_time , af.last_update_time as last_update_time 
, version_name, version_code
from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
unnest(Application) as af) ca
ON ca.deviceId = t3.refno
LEFT JOIN prj-prod-dataplatform.dap_ds_poweruser_playground.PH_Tonikbank_Application_Temp ptat
ON REGEXP_REPLACE(ca.package_name, r'[ ._]', '') = REGEXP_REPLACE(ptat.Package_Name, r'[ ._]', '')
where date(lmt.startApplyDateTime) >='2023-07-01'   ---- Please change the date as per your requirement. This is Loan Application Apply Date
-- and lmt.FSPD30_loancnt is not null
)
select brand, device__brand, count(distinct customerId) cntcust 
, (select count(distinct customerId) from base) totalcust
from base 
group by 1,2
;
"""

df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 12d93a37-0c14-4182-af5a-594b15460b18 successfully executed: 100%|██████████|

I0000 00:00:1727757613.023709   28868 ev_epoll1_linux.cc:125] grpc epoll fd: 59



Downloading: 100%|██████████|


I0000 00:00:1727757613.878081   28980 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1727757613.879162   28980 work_stealing_thread_pool.cc:269] WorkStealingThreadPoolImpl::Quiesce


In [13]:
df['share'] = round(df['cntcust']/df['totalcust'] *100, 4)

In [14]:
pd.set_option("Display.max_rows", None)
df.sort_values(by='cntcust', ascending = False)

,brand,device__brand,cntcust,totalcust,share
25,OPPO,OPPO,25396,117690,21.5787
17,vivo,vivo,21150,117690,17.9709
23,samsung,samsung,20369,117690,17.3073
68,realme,realme,16213,117690,13.776
74,Infinix,Infinix,10021,117690,8.5147
1,Redmi,Redmi,8761,117690,7.4441
63,TECNO,TECNO,5469,117690,4.647
38,HUAWEI,HUAWEI,3709,117690,3.1515
19,HONOR,HONOR,2998,117690,2.5474
53,POCO,POCO,2004,117690,1.7028


In [15]:
pd.set_option("Display.max_columns", None)
df.head()

,brand,device__brand,cntcust,totalcust,share
0,CUBOT,CUBOT,1,117690,0.0008
1,Redmi,Redmi,8761,117690,7.4441
2,benco,benco,3,117690,0.0025
3,EL,EL,2,117690,0.0017
4,asus,asus,94,117690,0.0799


In [16]:
df.to_csv(r"/home/jupyter/DS_Team/DC/App_Package_Categorization/App_Categorization/Data/DeviceManufacturer_20241001.csv", index = False)

In [ ]:
# first_install_time
## Last 5 digits of First Install Time are all 0



